In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [1]:
import os, shutil
import bs4 as bs
import requests
import datetime
import pandas as pd
import glob

root_dir = "/Users/trevorwiebe/Ktor/radar_backend/radar_data/"

In [ ]:
# Downloading csv file

def download_csv_files(folder_url, destination_folder, start_time, end_time):

  try:

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

    # Delete old files
    for filename in os.listdir(destination_folder):
      file_path = os.path.join(destination_folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))


    # Get the list of files in the folder
    response = requests.get(folder_url)
    data = bs.BeautifulSoup(response.text, "html.parser")

    csv_files = data.find_all("a", href=lambda href: href and href.endswith(".csv"))

    # Filter CSV files based on time range
    filtered_files = []
    for file_name in csv_files:
        csv_filename = file_name['href'].split('/')[-1]
        try:
            # Parse the filename to extract the creation or modification time
            file_time_str = csv_filename.split('_')[0]
            file_time = datetime.datetime.strptime(file_time_str, '%d-%m-%Y-%H%M')
            if start_time <= file_time <= end_time:
                filtered_files.append(file_name)
        except ValueError as e:
            # Handle parsing errors (e.g., invalid filename format)
            print(f"Error parsing filename: {csv_filename}, {e.args[0]}")

    for file_name in filtered_files:
      csv_url = file_name['href']  # Get the CSV file URL
      csv_filename = csv_url.split('/')[-1]  # Extract the filename
      link = folder_url + csv_filename
      destination_link = os.path.join(destination_folder, csv_filename)

      response = requests.get(link)
      with open(destination_link, 'wb') as f:
         f.write(response.content)

    print("Downloading finished, outcome unknown.")

  except requests.exceptions.RequestException as e:
    print(f"Error downloading files: {e}")



# Initiate download of files
folder_url = "http://69.48.179.226/csv_files/"
destination_folder = root_dir + "csv_files"
start_time = datetime.datetime(2024, 9, 15, 0, 40)
end_time = datetime.datetime(2024, 9, 15, 1, 10)

download_csv_files(folder_url, destination_folder, start_time, end_time)

In [86]:
# Combines multiple CSV files into one.
def combine_csv_files(input_folder, output_file):

    # Get a list of all CSV files in the input folder
    csv_files = glob.glob(input_folder + "/*.csv")

    # Initialize an empty list to store DataFrames
    dataframes = []

    # Iterate through each CSV file
    for file in csv_files:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file)

        df = df.rename(columns={'time': 'dateTime'})
        df = df.rename(columns={'unknown': 'reflectivity'})

        # Select the desired columns
        df = df[['dateTime', 'latitude', 'longitude', 'reflectivity']]

        # Append the DataFrame to the list
        dataframes.append(df)

    # Concatenate all DataFrames into one
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(output_file, index=False)

input_folder = root_dir + "csv_files"
output_file = root_dir + "data/combined_data.csv"

combine_csv_files(input_folder, output_file)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

pd.set_option('display.max_columns', None)
np.set_printoptions(precision=15)

data = pd.read_csv(root_dir + 'data/combined_data.csv')

In [3]:
# Setting device
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [4]:
# prepare data frame to be dateTime | latitude | longitude | reflectivity | reflectivity_1 to _15
from copy import deepcopy as dc

def prepare_dataframe_for_lstm(df, n_steps):
  df = dc(df)

  df['datetime'] = pd.to_datetime(df['dateTime'])

  # Encode 'datetime' as cyclical features (excluding day encoding)
  df['minute_sin'] = np.sin(2 * np.pi * df['datetime'].dt.minute / 60)
  df['minute_cos'] = np.cos(2 * np.pi * df['datetime'].dt.minute / 60)
  df['hour_sin'] = np.sin(2 * np.pi * df['datetime'].dt.hour / 24)
  df['hour_cos'] = np.cos(2 * np.pi * df['datetime'].dt.hour / 24)
  df['month_sin'] = np.sin(2 * np.pi * df['datetime'].dt.month / 12)
  df['month_cos'] = np.cos(2 * np.pi * df['datetime'].dt.month / 12)

  # Move new columns to the front
  new_columns = ['minute_sin', 'minute_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos']
  remaining_columns = [col for col in df.columns if col not in new_columns]
  df = df[new_columns + remaining_columns]

  df = df.drop(columns=['dateTime'])
  df = df.drop(columns=['datetime'])

  for i in range(1, n_steps+1):
    df[f'reflectivity_{i}'] = df['reflectivity'].shift(i)

  df.dropna(inplace=True)

  df = df[['reflectivity'] + [col for col in df.columns if col != 'reflectivity']]

  return df

lookback = 15
df = prepare_dataframe_for_lstm(data, lookback)

In [5]:
# remove rows where all -99
# List of the columns reflectivity_1 to reflectivity_15
reflectivity_columns = [f'reflectivity_{i}' for i in range(1, 16)]
reflectivity_columns.append('reflectivity')

# convert all the rows that are less than 0.0 to 0.0
df[reflectivity_columns] = df[reflectivity_columns].mask(df[reflectivity_columns] <= 0, 0)

# Remove rows where all values in the reflectivity columns are 0.0
no_zero_df = df[~(df[reflectivity_columns] == 0.0).all(axis=1)]

In [6]:
#Split into X and y, and shape correctly
X_df = no_zero_df.iloc[:, 7:]
y_df = no_zero_df.iloc[:, 0]

lat_lon = X_df.iloc[:, [0,1]].to_numpy()
time_series_data = X_df.iloc[:, 2:].to_numpy()
time_series_data = time_series_data.reshape(X_df.shape[0], 15, 1)
time_series_data[time_series_data < 15] = 15
lat_lon_repeated = np.repeat(lat_lon[:, np.newaxis, :], 15, axis=1)

X = np.concatenate([lat_lon_repeated, time_series_data], axis=2)
y = y_df.to_numpy().flatten()


In [7]:
# Create train, val and test splits

train_split = int(X.shape[0] * .8)
val_split = int(X.shape[0] * .9)

X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:val_split], y[train_split:val_split]
X_test, y_test = X[val_split:], y[val_split:]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((11815868, 15, 3),
 (11815868,),
 (1476984, 15, 3),
 (1476984,),
 (1476984, 15, 3),
 (1476984,))

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# Use this if starting from scratch
model1 = Sequential()
model1.add(InputLayer((lookback, 3)))
# LSTM layer with 64 units and dropout for regularization
model1.add(LSTM(64, return_sequences=False))  # return_sequences=False because we predict one value
model1.add(Dropout(0.2))  # Helps prevent overfitting

# Dense layer for additional feature extraction
model1.add(Dense(32, activation='relu'))  # Increased neurons for more complexity
model1.add(Dropout(0.2))  # More dropout

# Final output layer (predicting a single value)
model1.add(Dense(1, activation='linear'))

model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,521 (76.25 KB)

 Trainable params: 19,521 (76.25 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# model1 = load_model(root_dir + 'model1/model1.keras')
cp = ModelCheckpoint(root_dir + 'model/model4.keras', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [10]:
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, callbacks=[cp])

Epoch 1/2
369246/369246 ━━━━━━━━━━━━━━━━━━━━ 1466s 4ms/step - loss: 20.1755 - root_mean_squared_error: 4.4866 - val_loss: 16.5070 - val_root_mean_squared_error: 4.0629
Epoch 2/2
369246/369246 ━━━━━━━━━━━━━━━━━━━━ 1440s 4ms/step - loss: 16.9582 - root_mean_squared_error: 4.1180 - val_loss: 14.9306 - val_root_mean_squared_error: 3.8640


In [11]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', None)

In [12]:
test_predictions = model1.predict(X_test).flatten()
third_column = np.flip(X_test[:, :, X_test.shape[2]-1], axis=1)
X_test_strings = [' '.join(map(str, row)) for row in third_column]
test_results = pd.DataFrame(data={'Historical':X_test_strings, 'Actuals':y_test, 'Val Predictions':test_predictions,})
test_results.head(100)

46156/46156 ━━━━━━━━━━━━━━━━━━━━ 49s 1ms/step


,Historical,Actuals,Val Predictions
0,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,4.5,3.211029
1,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,3.3,3.179068
2,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,6.2,3.147737
3,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,1.5,3.117099
4,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,1.2,3.087013
5,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,1.3,3.057524
6,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,6.2,3.028703
7,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,2.2,3.000413
8,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,1.7,2.972699
9,15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0,1.8,2.945318
